# Helper

## Data Processing, Defining PyTorch Dataset and Transforms

In [1]:
from torch.utils.data import Dataset

In [2]:
class PatientDataSet(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.data_columns = self.df.columns[0:len(df.columns)-1]
        self.target_column = self.df.columns[len(df.columns)-1]
        self.transform = transform

    def __len__(self):
        return len(self.df.index)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        selected_rows = self.df.iloc[idx]

        data = selected_rows[self.data_columns]
        diagnosis = selected_rows[self.target_column]
        index = selected_rows.index

        sample = {'data':data, 'diagnosis':diagnosis}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def get_X(self):
        data = self.df[self.data_columns]
        return data

    def get_Y(self):
        diagnosis = self.df[self.target_column]
        return diagnosis

In [3]:
class GaussianNoise:
    def __init__(self, mean, std_dev, data_columns):
        self.mean = mean
        self.std_dev = std_dev
        self.data_columns = data_columns

    def __call__(self, sample):
        shape = sample['data'][self.data_columns].shape
        noise = np.random.normal(self.mean, self.std_dev, shape)

        data = sample['data']
        data_noisy = data.copy()
        data_noisy[self.data_columns] += noise

        return {'data':data, 'data_noisy':data_noisy, 'diagnosis':sample['diagnosis']}

In [4]:
class ToTorchFormat:
    def __call__(self, sample):
        data = sample['data'].astype('float64')
        data_formatted = torch.from_numpy(data.to_numpy()).float()
        diagnosis = sample['diagnosis'].astype('int64')
        diagnosis_formatted = torch.from_numpy(np.asarray(diagnosis))

        sample_new = {'data':data_formatted,
                      'diagnosis':diagnosis_formatted}
        if 'data_noisy' in sample.keys():
            data_noisy = sample['data_noisy'].astype('float64')
            data_noisy_formatted = torch.from_numpy(data_noisy.to_numpy()).float()
            sample_new['data_noisy'] = data_noisy_formatted

        return sample_new

In [5]:
def split_df(df, **kwargs):
  # split dateframe
    n_rows = kwargs.get('n_rows', None)
    randomize = kwargs.get('randomize', False)
    random_seed = kwargs.get('random_seed', None)
    train_test_split = kwargs.get('train_test_split', False)
    percent_split = kwargs.get('percent_split', None)
    validation_split = kwargs.get('validation_split', None)

    if n_rows:
        n_rows = min(n_rows, len(df.index)-1)

    if not randomize:
        if train_test_split:
            if percent_split:
                if n_rows:
                    split = int(percent_split*n_rows)
                    train_df = df.iloc[0:split]
                    test_df = df.iloc[split:n_rows]
                else:
                    split = int(percent_split*(len(df.index)-1))
                    train_df = df.iloc[0:split]
                    test_df = df.iloc[split:]
                if validation_split:
                    split = int(validation_split*(len(train_df.index)-1))
                    val_df = train_df.iloc[0:split]
                    train_df = train_df.iloc[split:]
            else:
                raise Exception("Wanted train test split, but no split was given. Must set 'percent_split' as an argument.")

            if not validation_split:
                return train_df, test_df
            else:
                return train_df, val_df, test_df

        else:
            if percent_split:
                if n_rows:
                    split = int(percent_split*n_rows)
                else:
                    split = int(percent_split*(len(df.index)-1))
                df = df.iloc[0:split]
            elif n_rows:
                df = df.iloc[0:nrows]

            return df

    else:
        if random_seed:
            np.random.seed(random_seed)

        if train_test_split:
            if percent_split:
                if n_rows:
                    split = int(percent_split*n_rows)
                    df_indices = np.random.choice(df.index, n_rows)
                else:
                    split = int(percent_split*(len(df.index)-1))
                    df_indices = np.random.choice(df.index, len(df.index))
                train_df = df.loc[df_indices[0:split]]
                test_df = df.loc[df_indices[split:]]

                if validation_split:
                    split = int(validation_split*(len(train_df.index)-1))
                    val_df = train_df.iloc[0:split]
                    train_df = train_df.iloc[split:]
            else:
                raise Exception("Wanted train test split, but no split was given. Must set 'percent_split' as an argument.")

            if not validation_split:
                return train_df, test_df
            else:
                return train_df, val_df, test_df

        else:
            if percent_split:
                if n_rows:
                    split = int(percent_split*n_rows)
                else:
                    split = int(percent_split*(len(df.index)-1))
                df_indices = np.random.choice(df.index, split)
                df = df.loc[df_indices]
            elif n_rows:
                df_indices = np.random.choice(df.index, n_rows)
                df = df.loc[df_indices]
            else:
                df_indices = np.random.choice(df.index, len(df.index))
                df = df.loc[df_indices]

            return df

In [6]:
def generate_datasets(csv_path, **kwargs):
  # Generating PyTorch Datasets from Data
    dataset_type = kwargs.get('dataset_type', 'RS')
    non_data_cols = kwargs.get('non_data_cols', [])

    kwargs['cols_to_ignore'] = non_data_cols

    if dataset_type == 'RS':
        df = load_corr_clinical(csv_path, **kwargs)
    elif dataset_type == 'DTI':
        df = load_DTI(csv_path, **kwargs)
    elif dataset_type == 'T1':
        df = load_T1(csv_path, **kwargs)
    else:
        raise ValueError("Invalid argument {} for kwarg 'type'. Only accepts 'RS', 'DTI', or 'T1'".format(dataset_type))

    dfs = split_df(df, **kwargs)

    if type(dfs) is tuple:
        if len(dfs) == 2:
            train_dataset = PatientDataSet(dfs[0], dataset_type=dataset_type, non_data_cols=non_data_cols)
            test_dataset = PatientDataSet(dfs[1], dataset_type=dataset_type, non_data_cols=non_data_cols)
            return train_dataset, test_dataset
        elif len(dfs) == 3:
            train_dataset = PatientDataSet(dfs[0], dataset_type=dataset_type, non_data_cols=non_data_cols)
            val_dataset = PatientDataSet(dfs[1], dataset_type=dataset_type, non_data_cols=non_data_cols)
            test_dataset = PatientDataSet(dfs[2], dataset_type=dataset_type, non_data_cols=non_data_cols)
            return train_dataset, val_dataset, test_dataset
    else:
        return PatientDataSet(dfs, dataset_type=dataset_type, non_data_cols=non_data_cols)

## Tracking and Storing Model Performance

In [7]:
import pickle
import os

In [8]:
class Log:
    def __init__(self, log_name='log.txt', log_path = '.', should_log=True, overwrite=True):
        self.log_name = log_name
        self.log_path = log_path
        self.curr_str = ""
        self.total_str = ""
        self.should_log = should_log
        self.overwrite = overwrite

    def enable(self):

        self.should_log = True

    def disable(self):
        self.should_log = False

    def clear(self):
        self.curr_str = ""
        self.total_str = ""

    def set_name(self, name):
        self.log_name = name

    def set_path(self, path):
        self.log_path = path

    def get_log(self):
        return self.total_str

    def print_log(self):
        print(self.total_str)

    def save(self):
        if self.should_log:
            fpath = os.path.join(self.log_path, self.log_name)
            if self.overwrite:
                with open(fpath, 'w+') as logfile:
                    logfile.write(self.total_str)
            else:
                with open(fpath, 'a') as logfile:
                    logfile.write(self.curr_str)
                self.curr_str = ""

    def print_and_log(self, *args, **kwargs):
        if self.should_log:
            new_str= ""
            for s in args:
                new_str += str(s)
            new_str += '\n'
            self.curr_str += new_str
            self.total_str += new_str
            print(*args, **kwargs)
        else:
            print(*args, **kwargs)
            return None


class Stats:
    def __init__(self, filename='stats', path='.', load=True):
        # Want to ensure stats files have .stt extension
        fname, fext = os.path.splitext(filename)
        if fext != ".stt" and fext != "":
            raise Exception("Filename '{}' does not have correct extension ('{}' instead of '.stt')".format(filename, fext))
        if fext != ".stt":
            self.filename = fname + ".stt"

        self.path = path

        filepath = os.path.join(self.path, self.filename)

        if load and os.path.exists(filepath):
            with open(filepath, 'rb') as pickle_file:
                self.stats = pickle.load(pickle_file)
        else:
            if load:
                print("Warning: You are trying to load a stats file '{}' that doesn't exist.".format(os.path.abspath(filepath)))
            self.stats = {}

    def clear_stat(self, statname):
        if statname in self.stats:
            self.stats[statname] = {}

    def delete_stat(self, statname):
        del stats[statname]

    def track_stat(self, statname, key, val):
        if statname not in self.stats:
            self.stats[statname] = {}
        self.stats[statname][key] = val

    def set_name(self, name):
        self.filename = name

    def set_path(self, path):
        self.path = path

    def save(self):
        filepath = os.path.join(self.path, self.filename)
        with open(filepath, 'wb') as pickle_file:
            pickle.dump(self.stats, pickle_file)

    def __getitem__(self, key):
        return self.stats[key]

## Generating, Loading, and Storing Models

In [9]:
import os
import warnings
import re

In [10]:
# Network naming stuff
network_ext = '.nn'
network_name_fstr = '{}_{}' + network_ext

models_path = 'vae_nets'


def get_model_path(model_name):
    return os.path.join(models_path, model_name)


def make_model_path(model_name):

    path = get_model_path(model_name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path


class NetworkMetadata:
    filename = 'netinfo.mtd'

    def __init__(self, model_name, input_size=None, hidden_size=None, latent_size=None, activations=None):
        self.model_name = model_name
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.activations = activations

    def set_data(input_size, hidden_size, latent_size, activations):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.activations = activations

    def save(self):
        model_path = get_model_path(self.model_name)
        metadata_path = os.path.join(model_path, NetworkMetadata.filename)
        with open(metadata_path, 'w+') as f:
            f.write('model_name: {}\n'.format(self.model_name))
            f.write('input_size: {}\n'.format(self.input_size))
            f.write('hidden_size: {}\n'.format(self.hidden_size))
            f.write('latent_size: {}\n'.format(self.latent_size))
            f.write('activations: {}\n'.format(', '.join(self.activations)))

    def load(self):
        model_path = get_model_path(self.model_name)
        metadata_path = os.path.join(model_path, NetworkMetadata.filename)
        with open(metadata_path) as f:
            for line in f.readlines():
                line = line.replace('\n', '')
                linesplit = line.split(': ')
                line_name = linesplit[0]
                line_data = linesplit[1]
                if line_name == 'input_size':
                    self.input_size = int(line_data)
                elif line_name == 'hidden_size':
                    self.hidden_size = int(line_data)
                elif line_name == 'latent_size':
                    self.latent_size = int(line_data)
                elif line_name == 'activations':
                    self.activations = line_data.split(', ')

    @staticmethod
    def exists(model_name):
        model_path = get_model_path(model_name)
        metadata_path = os.path.join(model_path, NetworkMetadata.filename)
        return os.path.exists(metadata_path)


def save_network(model, model_name, idx, log=None):
    '''
    Saves the specified model

    params:
        model: VAE
            The model to save
        model_name: str
            The name of the model to save to
        idx: int
            The current epoch at which this model is training
        log: Log (default = None)
            The Log to which to log the saving operation
    '''

    path = get_model_path(model_name)
    if not os.path.exists(path):
        raise Exception("Path for network '{}' does not exist, please call make_model_path first!".format(path))
    save_fpath = os.path.join(path, network_name_fstr.format(model_name, idx))
    torch.save(model.state_dict(), save_fpath)
    if not NetworkMetadata.exists(model_name):
        metadata = NetworkMetadata(model_name, model.input_size, model.hidden_size, model.latent_size, model.activations_str)
        metadata.save()
    if log:
        log.print_and_log("Saving model to ", save_fpath)
    else:
        print("Saving model to ", save_fpath)


def get_file_index(file):
    '''
    Used to get the indices of files that have a number at
    the end. (Used for getting epoch of model checkpoints)

    params:
        file: str
            The name of the file to get the index from

    return:
        int
            The index of the file
    '''
    filename, _ =  os.path.splitext(file)
    idx = re.findall('[0-9]+', filename)[-1]
    return int(idx)


def load_network(model_name, model=None, idx=None, log=None):
    '''
    Loads the model of the specified name's state into the specified model

    params:
        model_name: str
            The name of the model to load the state from
        model: VAE (default = None)
            If not None model to load the state to
            Otherwise, load model from metadata
        idx: int (default = None)
            The epoch from which to load the model
        log: Log (default = None)
            The Log to which to log the loading operation
    '''

    path = os.path.join("vae_nets", model_name)
    if not os.path.exists(path):
        raise Exception("Model by model_name '{}' does not exist!".format(model_name))
    files = [f for f in os.listdir(path) if model_name in f and '.nn' in f]
    if not files:
        raise Exception("Model directory for '{}' exists, but no saved networks found inside the directory!".format(model_name))
    if idx is None:
        files_sorted = sorted(files, key=lambda f : get_file_index(f), reverse=True)
        model_file = files_sorted[0]
    else:
        file_target = network_name_fstr.format(model_name, idx)
        if file_target in files:
            model_file = file_target
        else:
            raise Exception("Model directory for '{}' exists, but no network at epoch {} found inside the directory!".format(model_name, idx))

    model_path = os.path.join(path, model_file)

    if model is None:
        if NetworkMetadata.exists(model_name):
            metadata = NetworkMetadata(model_name)
            metadata.load()
            model = VAE(metadata.input_size, metadata.hidden_size, metadata.latent_size, metadata.activations)

    if model is None:
        raise Exception("No model was provided to load state into and metadata does not exist for model.")

    model.load_state_dict(torch.load(model_path))

    if log:
        log.print_and_log("Successfully loaded model {}.".format(os.path.abspath(model_path)))
    else:
        print("Successfully loaded model {}.".format(os.path.abspath(model_path)))

    return model, get_file_index(model_file)


def overwrite_network(model_name, epoch=None):
    '''
    Clears the model path for the specified model

    params:
        model_name: str
            The name of the model to overwrite
        epoch: int (default = None)
            If an epoch is not None, then only the checkpoints
            after that checkpoint are deleted
            Otherwise, the whole model is deleted

    return:
        bool
            Whether the model path exists or not
    '''

    path = get_model_path(model_name)
    if not os.path.exists(path):
        return False
    if not epoch:
        exts_to_check = [network_ext, '.mtd', '.stt', '.csv']
        for f in os.listdir(path):
            fpath = os.path.join(path, f)
            _, fext = os.path.splitext(fpath)
            if fext in exts_to_check:
                os.remove(fpath)
        return True
    else:
        exts_to_check = [network_ext]
        for f in os.listdir(path):
            fpath = os.path.join(path, f)
            _, fext = os.path.splitext(fpath)
            if fext in exts_to_check and get_file_index(f) > epoch:
                os.remove(fpath)
        return True

In [11]:
def rename_network(net_name_old, net_name_new):
    import warnings
    model_path_old = get_model_path(net_name_old)
    model_path_new = get_model_path(net_name_new)
    if not os.path.exists(model_path_new):
        os.rename(model_path_old, model_path_new)
    else:
        warnings.warn("Model: {} already exists. Can't rename {} to {}".format(net_name_new, net_name_old, net_name_new))
        return
    net_meta = NetworkMetadata(net_name_new)
    net_meta.load()
    net_meta.model_name = net_name_new
    net_meta.save()
    for f in os.listdir(model_path_new):
        fname, ext = os.path.splitext(f)
        if(net_name_old in fname):
            f_old = os.path.join(model_path_new, "{}{}".format(fname, ext))
            f_new = os.path.join(model_path_new, "{}{}".format(fname.replace(net_name_old, net_name_new), ext))
            os.rename(f_old, f_new)

## VAE and Loss

In [12]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [13]:
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size=400, latent_size=20, activations=('tanh', 'selu', 'tanh', 'tanh')):
        super(VAE, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.fc21 = nn.Linear(self.hidden_size, self.latent_size)
        self.fc22 = nn.Linear(self.hidden_size, self.latent_size)
        self.fc3 = nn.Linear(self.latent_size, self.hidden_size)
        self.fc4 = nn.Linear(self.hidden_size, self.input_size)
        self.activations_str = activations
        activation_list = []
        for activation in activations:
            if activation == 'selu':
                activation_list.append(torch.selu)
            elif activation == 'relu':
                activation_list.append(torch.relu)
            elif activation == 'sigmoid':
                activation_list.append(torch.sigmoid)
            elif activation == 'tanh':
                activation_list.append(torch.tanh)
            else:
                activation_list.append(lambda x : x)
        self.activations = tuple(activation_list)

    def encode(self, x):
        h1 = self.activations[0](self.fc1(x))
        return self.activations[1](self.fc21(h1)), self.activations[1](self.fc22(h1))

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = self.activations[2](self.fc3(z))
        return self.activations[3](self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


# Reconstruction loss function
reconstruction_function = nn.MSELoss(reduction='none')


def loss_function(recon_x, x, mu, logvar, terms=('MSE', 'KLD'), beta=1):
    loss = 0

    if 'MSE' in terms:
        # MSE = 1/n*sum(x_recon - x)^2
        # Need to sum over feature size, and mean over batch size
        SE = reconstruction_function(recon_x, x)
        MSE = torch.mean(torch.sum(SE, dim=1), dim = 0)
        loss += MSE
        ##print('MSE =', MSE.data.item())

    if 'KLD' in terms:
        KLD = torch.mean(-0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim = 1), dim = 0)
        loss += beta * KLD
        ##print('KLD =', KLD.data.item())

    return loss

## Train and Validation Loop

In [14]:
def train_net(net_name, dataset, **kwargs):
    activations = kwargs.get('activations', ('tanh', 'selu', 'tanh', 'tanh'))
    hidden_size = kwargs.get('hidden_size', 400)
    latent_size = kwargs.get('latent_size', 20)
    loss_terms = kwargs.get('loss_terms', ('MSE', 'KLD'))
    _beta = kwargs.get('beta', 1)
    warm_start = kwargs.get('warm_start', 100)
    num_epochs = kwargs.get('num_epochs', 100)
    until = kwargs.get('until', None)
    batch_size = kwargs.get('batch_size', 128)
    shuffle = kwargs.get('shuffle', True)
    learning_rate = kwargs.get('learning_rate', 1e-3)
    clip_val = kwargs.get('clip_val', None)
    l2 = kwargs.get('l2', 1e-2)
    load_net = kwargs.get('load_net', False)
    net_epoch = kwargs.get('net_epoch', None)
    overwrite = kwargs.get('overwrite', False)
    save_net_freq = kwargs.get('save_net_freq', 100)
    _validation = kwargs.get('validation', None)
    print_net_freq = kwargs.get('print_net_freq', 100)
    should_log = kwargs.get('should_log', True)
    overwrite_log = kwargs.get('overwrite_log', False)
    load_stats = kwargs.get('load_stats', True)
    stats_list = kwargs.get('stats_list', ["epoch", "loss", "loss_per_feature"])


    model_path = make_model_path(net_name)

    log = Log(log_name=net_name+'_training_log.txt', should_log=should_log, log_path=get_model_path(net_name), overwrite=overwrite_log)

    log.print_and_log("Training Network '{}'".format(net_name))

    idx_offset = 0

    if not load_net or not NetworkMetadata.exists(net_name):
        model = VAE(input_size=len(dataset.data_columns), hidden_size=hidden_size, latent_size=latent_size, activations=activations)

    # Loading the model if desired
    # Also handles loading stats
    if load_net:
        if not NetworkMetadata.exists(net_name):
            warnings.warn('Requested to load network, but no metadata exists. Loading model state into newly generated model.')
            _, idx_offset = load_network(net_name, model, idx=net_epoch, log=log)
        else:
            model, idx_offset = load_network(net_name, None, idx=net_epoch, log=log)
        idx_offset += 1

        if load_stats:
            stats = Stats(load=True, path=model_path)
        else:
            stats = Stats(load=False, path=model_path)
    else:
        stats = Stats(load=False, path=model_path)

    if overwrite:
        if load_net:
            overwrite_network(net_name, idx_offset)
        else:
            overwrite_network(net_name)

    if torch.cuda.is_available():
        model.cuda()

    if l2:
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2)
    else:
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    for epoch in range(num_epochs):
        if until and (epoch + idx_offset >= until):
            break
        log.print_and_log("Train Epoch:", epoch + idx_offset)
        model.train()
        train_loss = 0
        for batch_idx, sample_batch in enumerate(dataloader):
            data_noisy = sample_batch['data_noisy']
            data = sample_batch['data']
            data_noisy = Variable(data_noisy)
            if torch.cuda.is_available():
                data_noisy = data_noisy.cuda()
                data = data.cuda()
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data_noisy)
            # The KLD weight for warm start
            if (warm_start is not None) and (warm_start > 0):
                beta = min((epoch+idx_offset)/warm_start*_beta, _beta)
            else:
                beta = _beta
            loss = loss_function(recon_batch, data, mu, logvar, terms=loss_terms, beta=beta)
            loss.backward()
            curr_loss = loss.data.item()
            train_loss += curr_loss
            # For clipping the loss
            if clip_val:
                clip_grad_norm_(model.parameters(), clip_val)
            optimizer.step()

            per_feature_loss = curr_loss / model.input_size

            if print_net_freq and print_net_freq > 0:
                if batch_idx % print_net_freq == 0:
                    log.print_and_log('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tLoss Per Feature {:.6f}'.format(
                        epoch + idx_offset,
                        batch_idx * len(data_noisy),
                        len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                        curr_loss, per_feature_loss))
            else:
                log.print_and_log('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tLoss Per Feature: {:.6f}'.format(
                epoch + idx_offset,
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                curr_loss, per_feature_loss))

        avg_train_loss = train_loss / (batch_idx + 1)
        avg_per_feature_loss = avg_train_loss / model.input_size
        log.print_and_log('====> Epoch: {} Average Loss: {:.6f}\tAverage Loss Per Feature {:.6f}'.format(
            epoch + idx_offset, avg_train_loss, avg_per_feature_loss))

        if 'epoch' in stats_list:
            stats.track_stat('epoch', epoch, epoch)

        if 'loss' in stats_list:
            stats.track_stat('loss', epoch, avg_train_loss)

        if 'loss_per_feature' in stats_list:
            stats.track_stat('loss_per_feature', epoch, avg_per_feature_loss)

        if save_net_freq and save_net_freq > 0:
            if (epoch + idx_offset) % save_net_freq == 0:
                save_network(model, net_name, epoch + idx_offset, log)
                stats.save()

        if _validation:
            if isinstance(_validation, (list, tuple)):
                val_loss = []
                val_loss_per_feature = []
                for val_ds in _validation:
                    val_loss_curr, val_loss_per_feature_curr = validation(net_name, val_ds, model=model, log=log)
                    val_loss.append(val_loss_curr)
                    val_loss_per_feature.append(val_loss_per_feature_curr)

            else:
                val_loss, val_loss_per_feature = validation(net_name, _validation, model=model, log=log)

            if 'validation_loss' in stats_list:
                stats.track_stat('validation_loss', epoch, val_loss)

            if 'validation_loss_per_feature' in stats_list:
                stats.track_stat('validation_loss_per_feature', epoch, val_loss_per_feature)

            if save_net_freq and save_net_freq > 0:
                if (epoch + idx_offset) % save_net_freq == 0:
                    stats.save()

        log.save()

    # Only save the final epoch if our network wasn't loaded past the until
    # epoch. Otherwise, there is no point in saving.
    if (until and (idx_offset < until)) or not until:
        save_network(model, net_name, epoch + idx_offset, log)
        stats.save()

    log.save()

    return epoch + idx_offset


In [15]:
def validation(net_name, dataset, **kwargs):
    loss_terms = kwargs.get('loss_terms', ('MSE', 'KLD'))
    batch_size = kwargs.get('batch_size', 64)
    shuffle = kwargs.get('shuffle', False)
    use_noisy = kwargs.get('use_noisy', True)
    net_epoch = kwargs.get('net_epoch', None)
    model = kwargs.get('model', None)
    log = kwargs.get('log', None)
    should_log = kwargs.get('should_log', True)
    overwrite_log = kwargs.get('overwrite_log', False)

    if not log:
        log = Log(log_name=net_name+'_validation_log.txt', should_log=should_log, log_path=get_model_path(net_name), overwrite=overwrite_log)

    log.print_and_log("Performing validation on model '{}'".format(net_name))

    if not model:
        model, _ = load_network(net_name, None, idx=net_epoch)
    if torch.cuda.is_available():
        model.cuda()
    model.eval()

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    test_loss = 0
    for batch_idx, sample_batch in enumerate(dataloader):
        data = sample_batch['data']
        if use_noisy:
            data_noisy = sample_batch['data_noisy']
            data_input = Variable(data_noisy)
        else:
            data_input = Variable(data)

        if torch.cuda.is_available():
            data_input = data_input.cuda()
            data = data.cuda()

        recon_batch, mu, logvar = model(data_input)

        loss = loss_function(recon_batch, data, mu, logvar, terms=loss_terms)

        curr_loss = loss.data.item()

        test_loss += curr_loss

        log.print_and_log('[{}/{} ({:.0f}%)]\tLoss: {:.6f}\tLoss Per Feature {:.6f}'.format(
                batch_idx * len(data_input),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                curr_loss, curr_loss / model.input_size))

        log.save()

    avg_loss = test_loss / (batch_idx + 1)
    avg_per_feature_loss = avg_loss / model.input_size

    log.print_and_log('Average Loss: {:.6f}\tAverage Loss Per Feature {:.6f}'.format(avg_loss, avg_per_feature_loss))

    log.save()

    return avg_loss, avg_per_feature_loss

## Latent Extraction and Loading

In [16]:
def extract_latents(net_name, dataset, **kwargs):
    batch_size = kwargs.get('batch_size', 64)
    use_noisy = kwargs.get('use_noisy', False)
    return_df = kwargs.get('return_df', True)
    fname_tag = kwargs.get('fname_tag', None)
    net_epoch = kwargs.get('net_epoch', None)
    latent_fpath = kwargs.get('file_path', None)

    model, _ = load_network(net_name, None, idx=net_epoch)

    if torch.cuda.is_available():
        model.cuda()
    model.eval()

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    latent_df = None

    print('Extracting latents...')

    for batch_idx, sample_batch in enumerate(dataloader):

        if use_noisy:
            data_noisy = sample_batch['data_noisy']
            data_input = Variable(data_noisy)
        else:
            data = sample_batch['data']
            data_input = Variable(data)

        if torch.cuda.is_available():
            data_input = data_input.cuda()

        # print('[{}/{} ({:.0f}%)]'.format(
        #         batch_idx * len(data_input),
        #         len(dataloader.dataset), 100. * batch_idx / len(dataloader)))

        mu, logvar = model.encode(data_input)

        mu = mu.cpu().detach().numpy()
        logvar = logvar.cpu().detach().numpy()

        latent_arr = np.concatenate((mu, logvar), axis=1)

        latent_colnames = ['mu_{}'.format(i) for i in range(mu.shape[1])] + ['logvar_{}'.format(i) for i in range(logvar.shape[1])]

        curr_df = pd.DataFrame(sample_batch['diagnosis'])
        temp_df = pd.DataFrame(data=latent_arr, columns=latent_colnames)
        curr_df[latent_colnames] = temp_df

        if latent_df is None:
            latent_df = curr_df
        else:
            latent_df = pd.concat([latent_df, curr_df], ignore_index=True)

    print('done.')

    if latent_fpath is None:
        model_path = get_model_path(net_name)

        latent_fpath = os.path.join(model_path, net_name)

        if fname_tag:
            latent_fpath += '_{}'.format(fname_tag)
        if net_epoch:
            latent_fpath += '_{}'.format(net_epoch)

        latent_fpath += '_latents.csv'

    latent_df.to_csv(latent_fpath, index=False)

    print("Saving latents to ", os.path.abspath(latent_fpath))

    if return_df:
        return latent_df

    else:
        return None

In [17]:
def load_latents(net_name, fname_tag=None, net_epoch=None):

    model_path = get_model_path(net_name)

    latent_fpath = os.path.join(model_path, net_name)

    if fname_tag:
        latent_fpath += '_{}'.format(fname_tag)
    if net_epoch:
        latent_fpath += '_{}'.format(net_epoch)

    latent_fpath += '_latents.csv'

    print("Reading latents from ", os.path.abspath(latent_fpath))

    df = pd.read_csv(latent_fpath)

    return df

## Classifier

In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from numpy import sqrt
from numpy import argmax
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from matplotlib import pyplot

def classifiers(X_train, X_test, y_train, y_test, K = 5, verbose = 1):

    if K > 0:
        clfs = [
            SVC(),
            RandomForestClassifier()
        ]

        acc_dict = dict()

        for clf in clfs:

            if clf.__class__.__name__ == 'SVC':
                hyperparams = {
                    'C': [0.01,0.1,1,10,50],
                    'gamma': ['auto','scale'],
                    'kernel': ['linear','rbf']
                }
            if clf.__class__.__name__ == 'RandomForestClassifier':
                hyperparams = {
                    'n_estimators': [50,100,600,900],
                    'max_features': ['auto', 'sqrt'],
                    'max_depth':[3,5],
                    'criterion': ['gini', 'entropy'],
                    'n_jobs':[-1]
                }

            cross_valid = StratifiedKFold(n_splits = K, shuffle = True, random_state = 0)
            grid_search = GridSearchCV(clf, param_grid = hyperparams, cv = cross_valid, scoring='roc_auc')
            grid_search.fit(X_train, y_train)
            PARAM, score = grid_search.best_params_, grid_search.best_score_

            if verbose != -1:
                print('==== clf: ', clf.__class__.__name__,
                      '\n best parameter: ', PARAM,
                      '\n best auc: ', score)

            if clf.__class__.__name__ == 'SVC':
                model = SVC(C = PARAM['C'],
                            gamma = PARAM['gamma'],
                            kernel = PARAM['kernel'],
                            probability=True
                            )
            if clf.__class__.__name__ == 'RandomForestClassifier':
                model = RandomForestClassifier(n_estimators = PARAM['n_estimators'],
                                               max_features = PARAM['max_features'],
                                               max_depth = PARAM['max_depth'],
                                               criterion = PARAM['criterion'],
                                               n_jobs = PARAM['n_jobs']
                                               )
            model.fit(X_train, y_train)
            pred_labels = model.predict(X_test)
            pred_labels_train = model.predict(X_train)
            acc_dict[str(clf.__class__.__name__)] = accuracy_score(y_test, pred_labels)

            if verbose != -1:
                tn, fp, fn, tp = confusion_matrix(y_train, pred_labels_train).ravel()
                fpr_train, tpr_train, _ = roc_curve(y_train, pred_labels_train)

                print('train sensitivity: ', tp/(tp+fn))
                print('train specificity: ', tn/(tn+fp))
                print(' train acc: ', accuracy_score(y_train, pred_labels_train))
                print(' train auc: ', metrics.auc(fpr_train, tpr_train))

                # threshold-moving for imbalanced clf
                pred_labels_proba = model.predict_proba(X_test)[:, 1]
                fpr, tpr, thres = roc_curve(y_test, pred_labels_proba)
                gmeans = sqrt(tpr * (1-fpr))
                ix = argmax(gmeans)
                print('Best Threshold=%f, G-Mean=%.3f' % (thres[ix], gmeans[ix]))
                pyplot.plot([0,1], [0,1], linestyle='--', label='No Model')
                pyplot.plot(fpr, tpr, marker='.', label='Clf')
                pyplot.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
                pyplot.xlabel('False Positive Rate')
                pyplot.ylabel('True Positive Rate')
                pyplot.legend()
                pyplot.show()

                print('test sensitivity after thres tuning: ', tpr[ix])
                print('test specificity after thres tuning: ', 1-fpr[ix])
                pred_labels_thres = (pred_labels_proba >= thres[ix]).astype(bool)
                print(' test acc after thres tuning: ', accuracy_score(y_test, pred_labels_thres))
                print(' test auc after thres tuning: ', metrics.auc(fpr, tpr))

    return(acc_dict)

# Data Processing

In [19]:
import os
import random
import numpy as np
import pandas as pd
from torchvision import transforms
from sklearn.model_selection import train_test_split

In [20]:
# datapath = '../data'
# data = np.load(os.path.join(datapath, 'craddock_cor_data.npy'))
# label = np.load(os.path.join(datapath, 'autism_labels.npy'))

# atlas_want = 'craddock_scorr_mean'
atlas_want = 'power_2011'

qc = np.load(os.path.join('data', 'autism_qc.npy'))

# get all data & demo info
label = np.load(os.path.join('data', 'autism_labels.npy'))
label_qc = label[qc ==1]

site = np.load(os.path.join('data', 'autism_site.npy'))
site_qc = site[qc ==1]

age = np.load(os.path.join('data', 'autism_age.npy'))
age_qc = age[qc ==1]

sex = np.load(os.path.join('data', 'autism_sex.npy'), allow_pickle=True)
sex[sex =='F'] = 0
sex[sex =='M'] = 1
sex = sex.astype(str).astype(int)
sex_qc = sex[qc ==1]

data = np.load(os.path.join('data', 'fmri_'+atlas_want+'_cor.npy'))
data_qc = data[qc ==1,:]

# if combat
data_adj_qc = np.load(os.path.join('data', 'fmri_'+atlas_want+'_cor_adj.npy'))
data_qc = data_adj_qc

In [21]:
random.seed(10)

# split test
test_i = np.array(random.sample(range(len(data_qc)), k=int(len(data_qc)*0.2)))
test_data = data_qc[test_i,:]
test_sex = sex_qc[test_i]
test_age = age_qc[test_i]
test_label = label_qc[test_i]

train_val_mask = np.ones(len(data_qc), bool)
train_val_mask[test_i] = 0
train_val_data = data_qc[train_val_mask,:]
train_val_sex = sex_qc[train_val_mask]
train_val_age = age_qc[train_val_mask]
train_val_label = label_qc[train_val_mask]

# split val
val_i = np.array(random.sample(range(len(train_val_data)), k=int(len(train_val_data)*0.2)))
val_data = train_val_data[val_i,:]
val_sex = train_val_sex[val_i]
val_age = train_val_age[val_i]
val_label = train_val_label[val_i]

train_mask = np.ones(len(train_val_data), bool)
train_mask[val_i] = 0
train_data = train_val_data[train_mask,:]
train_sex = train_val_sex[train_mask]
train_age = train_val_age[train_mask]
train_label = train_val_label[train_mask]

# train_data, test_data, train_label, test_label = train_test_split(data_qc, label_qc, train_size = 0.8)
# train_data, val_data, train_label, val_label = train_test_split(train_data, train_label, train_size = 0.8)

# transform the train data to pytorch format
trainset = np.hstack((train_data, train_label.reshape((train_data.shape[0],1))))
train_dataset = PatientDataSet(pd.DataFrame(trainset))
trans = transforms.Compose([GaussianNoise(0,0.1,train_dataset.data_columns),ToTorchFormat()])
train_dataset.transform = trans
# transform the val data to pytorch format
valset = np.hstack((val_data, val_label.reshape((val_data.shape[0],1))))
val_dataset = PatientDataSet(pd.DataFrame(valset))
trans = transforms.Compose([GaussianNoise(0,0.1,val_dataset.data_columns),ToTorchFormat()])
val_dataset.transform = trans
# transform the test data to pytorch format
testset = np.hstack((test_data, test_label.reshape((test_data.shape[0],1))))
test_dataset = PatientDataSet(pd.DataFrame(testset))
trans = transforms.Compose([GaussianNoise(0,0.1,test_dataset.data_columns),ToTorchFormat()])
test_dataset.transform = trans

# Model Training

In [22]:
epochs = 1000
hidden_size = 80
latent_size = 5
activations = ('tanh', 'selu', 'tanh', 'tanh')
l2_rate = 0.1
learning_rate = 1e-5

# try smaller learning rates
net_name = atlas_want+'_combat_h{}_l{}_r{}_ep{}'.format(hidden_size, latent_size, int(l2_rate*100), epochs)
stats_list = ["epoch", "loss", "loss_per_feature", "validation_loss", "validation_loss_per_feature"]

In [23]:
train_net(net_name, train_dataset, num_epochs=epochs, until=epochs,
          hidden_size=hidden_size, latent_size=latent_size,
          activations=activations, l2=l2_rate, learning_rate=learning_rate,
          load_net=False, overwrite=True, overwrite_log=True,
          stats_list=stats_list, validation=[val_dataset])

print("====", net_name)
extract_latents(net_name, train_dataset, fname_tag='train')
extract_latents(net_name, val_dataset, fname_tag='val')
extract_latents(net_name, test_dataset, fname_tag='test')

Training Network 'power_2011_combat_h80_l5_r10_ep1000'
Train Epoch: 0
Train Epoch: 0 [0/660 (0%)]	Loss: 4152.946289	Loss Per Feature 0.118723
====> Epoch: 0 Average Loss: 4077.741862	Average Loss Per Feature 0.116574
Saving model to  vae_nets/power_2011_combat_h80_l5_r10_ep1000/power_2011_combat_h80_l5_r10_ep1000_0.nn
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 3851.283203	Loss Per Feature 0.110100
[64/164 (33%)]	Loss: 3832.582031	Loss Per Feature 0.109565
[72/164 (67%)]	Loss: 3883.833740	Loss Per Feature 0.111030
Average Loss: 3855.899658	Average Loss Per Feature 0.110232
Train Epoch: 1
Train Epoch: 1 [0/660 (0%)]	Loss: 3818.747070	Loss Per Feature 0.109169
====> Epoch: 1 Average Loss: 3732.655843	Average Loss Per Feature 0.106708
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 3583.574463	Loss Per Feature 0.102446
[64/164 (33%)]	Loss: 3585.137451	Loss Per Feature 0.102491
[72/164 (67%)]	Loss: 3640.924

[64/164 (33%)]	Loss: 3074.250000	Loss Per Feature 0.087886
[72/164 (67%)]	Loss: 2995.922119	Loss Per Feature 0.085647
Average Loss: 3093.487305	Average Loss Per Feature 0.088436
Train Epoch: 18
Train Epoch: 18 [0/660 (0%)]	Loss: 2962.718750	Loss Per Feature 0.084698
====> Epoch: 18 Average Loss: 3032.311483	Average Loss Per Feature 0.086687
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 3061.901123	Loss Per Feature 0.087533
[64/164 (33%)]	Loss: 2956.605469	Loss Per Feature 0.084523
[72/164 (67%)]	Loss: 3118.791260	Loss Per Feature 0.089159
Average Loss: 3045.765951	Average Loss Per Feature 0.087072
Train Epoch: 19
Train Epoch: 19 [0/660 (0%)]	Loss: 3036.045166	Loss Per Feature 0.086794
====> Epoch: 19 Average Loss: 3086.012451	Average Loss Per Feature 0.088222
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 3098.423584	Loss Per Feature 0.088577
[64/164 (33%)]	Loss: 3010.759033	Loss Per Feature 0.086071
[72

====> Epoch: 35 Average Loss: 2802.554728	Average Loss Per Feature 0.080119
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 2815.933594	Loss Per Feature 0.080501
[64/164 (33%)]	Loss: 2842.676025	Loss Per Feature 0.081266
[72/164 (67%)]	Loss: 2828.658203	Loss Per Feature 0.080865
Average Loss: 2829.089274	Average Loss Per Feature 0.080877
Train Epoch: 36
Train Epoch: 36 [0/660 (0%)]	Loss: 2832.672119	Loss Per Feature 0.080980
====> Epoch: 36 Average Loss: 2808.035116	Average Loss Per Feature 0.080275
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 2900.320801	Loss Per Feature 0.082914
[64/164 (33%)]	Loss: 2743.141602	Loss Per Feature 0.078420
[72/164 (67%)]	Loss: 2752.660645	Loss Per Feature 0.078692
Average Loss: 2798.707682	Average Loss Per Feature 0.080009
Train Epoch: 37
Train Epoch: 37 [0/660 (0%)]	Loss: 2780.888428	Loss Per Feature 0.079499
====> Epoch: 37 Average Loss: 2863.996623	Average Loss Per Fea

Train Epoch: 53 [0/660 (0%)]	Loss: 2632.773926	Loss Per Feature 0.075265
====> Epoch: 53 Average Loss: 2617.563314	Average Loss Per Feature 0.074830
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 2729.754883	Loss Per Feature 0.078038
[64/164 (33%)]	Loss: 2602.352783	Loss Per Feature 0.074395
[72/164 (67%)]	Loss: 2632.970703	Loss Per Feature 0.075271
Average Loss: 2655.026123	Average Loss Per Feature 0.075901
Train Epoch: 54
Train Epoch: 54 [0/660 (0%)]	Loss: 2589.314209	Loss Per Feature 0.074023
====> Epoch: 54 Average Loss: 2619.495931	Average Loss Per Feature 0.074886
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 2652.338867	Loss Per Feature 0.075824
[64/164 (33%)]	Loss: 2665.932861	Loss Per Feature 0.076213
[72/164 (67%)]	Loss: 2572.243408	Loss Per Feature 0.073535
Average Loss: 2630.171712	Average Loss Per Feature 0.075191
Train Epoch: 55
Train Epoch: 55 [0/660 (0%)]	Loss: 2687.994629	Loss Per Featur

[64/164 (33%)]	Loss: 2466.622803	Loss Per Feature 0.070515
[72/164 (67%)]	Loss: 2513.077637	Loss Per Feature 0.071843
Average Loss: 2493.929036	Average Loss Per Feature 0.071296
Train Epoch: 71
Train Epoch: 71 [0/660 (0%)]	Loss: 2507.547852	Loss Per Feature 0.071685
====> Epoch: 71 Average Loss: 2462.473022	Average Loss Per Feature 0.070397
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 2534.833252	Loss Per Feature 0.072465
[64/164 (33%)]	Loss: 2456.469238	Loss Per Feature 0.070225
[72/164 (67%)]	Loss: 2558.397217	Loss Per Feature 0.073139
Average Loss: 2516.566569	Average Loss Per Feature 0.071943
Train Epoch: 72
Train Epoch: 72 [0/660 (0%)]	Loss: 2479.785400	Loss Per Feature 0.070892
====> Epoch: 72 Average Loss: 2542.019084	Average Loss Per Feature 0.072671
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 2557.263184	Loss Per Feature 0.073106
[64/164 (33%)]	Loss: 2415.991943	Loss Per Feature 0.069068
[72

...

Train Epoch: 944 [0/660 (0%)]	Loss: 1073.356567	Loss Per Feature 0.030685
====> Epoch: 944 Average Loss: 1071.568400	Average Loss Per Feature 0.030634
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 1094.827148	Loss Per Feature 0.031299
[64/164 (33%)]	Loss: 1075.091064	Loss Per Feature 0.030734
[72/164 (67%)]	Loss: 1072.936646	Loss Per Feature 0.030673
Average Loss: 1080.951619	Average Loss Per Feature 0.030902
Train Epoch: 945
Train Epoch: 945 [0/660 (0%)]	Loss: 1090.860107	Loss Per Feature 0.031185
====> Epoch: 945 Average Loss: 1088.299377	Average Loss Per Feature 0.031112
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 1094.773804	Loss Per Feature 0.031297
[64/164 (33%)]	Loss: 1077.401245	Loss Per Feature 0.030800
[72/164 (67%)]	Loss: 1075.238892	Loss Per Feature 0.030739
Average Loss: 1082.471313	Average Loss Per Feature 0.030945
Train Epoch: 946
Train Epoch: 946 [0/660 (0%)]	Loss: 1061.651489	Loss Per

[64/164 (33%)]	Loss: 1077.335205	Loss Per Feature 0.030799
[72/164 (67%)]	Loss: 1076.096680	Loss Per Feature 0.030763
Average Loss: 1083.479858	Average Loss Per Feature 0.030974
Train Epoch: 962
Train Epoch: 962 [0/660 (0%)]	Loss: 1038.746338	Loss Per Feature 0.029695
====> Epoch: 962 Average Loss: 1057.224172	Average Loss Per Feature 0.030224
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 1093.549805	Loss Per Feature 0.031262
[64/164 (33%)]	Loss: 1076.266357	Loss Per Feature 0.030768
[72/164 (67%)]	Loss: 1072.237671	Loss Per Feature 0.030653
Average Loss: 1080.684611	Average Loss Per Feature 0.030894
Train Epoch: 963
Train Epoch: 963 [0/660 (0%)]	Loss: 1058.602661	Loss Per Feature 0.030263
====> Epoch: 963 Average Loss: 1080.135132	Average Loss Per Feature 0.030879
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 1097.920776	Loss Per Feature 0.031387
[64/164 (33%)]	Loss: 1074.495850	Loss Per Feature 0.0307

Train Epoch: 979 [0/660 (0%)]	Loss: 1106.668335	Loss Per Feature 0.031637
====> Epoch: 979 Average Loss: 1058.762238	Average Loss Per Feature 0.030268
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 1092.565063	Loss Per Feature 0.031234
[64/164 (33%)]	Loss: 1073.316162	Loss Per Feature 0.030684
[72/164 (67%)]	Loss: 1074.409546	Loss Per Feature 0.030715
Average Loss: 1080.096924	Average Loss Per Feature 0.030878
Train Epoch: 980
Train Epoch: 980 [0/660 (0%)]	Loss: 1045.515259	Loss Per Feature 0.029889
====> Epoch: 980 Average Loss: 1056.915009	Average Loss Per Feature 0.030215
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 1094.748047	Loss Per Feature 0.031296
[64/164 (33%)]	Loss: 1074.970825	Loss Per Feature 0.030731
[72/164 (67%)]	Loss: 1072.899902	Loss Per Feature 0.030672
Average Loss: 1080.872925	Average Loss Per Feature 0.030900
Train Epoch: 981
Train Epoch: 981 [0/660 (0%)]	Loss: 1078.071533	Loss Per

[64/164 (33%)]	Loss: 1071.570923	Loss Per Feature 0.030634
[72/164 (67%)]	Loss: 1079.983154	Loss Per Feature 0.030874
Average Loss: 1080.890706	Average Loss Per Feature 0.030900
Train Epoch: 997
Train Epoch: 997 [0/660 (0%)]	Loss: 1063.259521	Loss Per Feature 0.030396
====> Epoch: 997 Average Loss: 1073.344849	Average Loss Per Feature 0.030685
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 1091.201294	Loss Per Feature 0.031195
[64/164 (33%)]	Loss: 1071.028198	Loss Per Feature 0.030618
[72/164 (67%)]	Loss: 1071.958252	Loss Per Feature 0.030645
Average Loss: 1078.062581	Average Loss Per Feature 0.030819
Train Epoch: 998
Train Epoch: 998 [0/660 (0%)]	Loss: 1080.144409	Loss Per Feature 0.030879
====> Epoch: 998 Average Loss: 1067.672770	Average Loss Per Feature 0.030522
Performing validation on model 'power_2011_combat_h80_l5_r10_ep1000'
[0/164 (0%)]	Loss: 1091.270386	Loss Per Feature 0.031197
[64/164 (33%)]	Loss: 1071.548950	Loss Per Feature 0.0306

,0,mu_0,mu_1,mu_2,mu_3,mu_4,logvar_0,logvar_1,logvar_2,logvar_3,logvar_4
0,1,2.032248,0.460174,-0.219195,-0.735557,1.392282,-1.679043,-1.675820,-1.683294,-1.671181,-1.708219
1,0,1.627486,0.340415,-0.185212,-0.683483,1.127664,-1.675070,-1.668256,-1.682736,-1.669326,-1.706204
2,0,1.927703,0.454868,-0.244438,-0.727106,1.322784,-1.679528,-1.672643,-1.683694,-1.669613,-1.708978
3,0,1.902535,0.433991,-0.237575,-0.722412,1.298956,-1.677323,-1.673172,-1.683109,-1.670379,-1.707968
4,1,1.795206,0.417541,-0.229805,-0.695674,1.226844,-1.674416,-1.671792,-1.684750,-1.667587,-1.706086
...,...,...,...,...,...,...,...,...,...,...,...
200,0,2.005051,0.448928,-0.229037,-0.748172,1.392352,-1.677420,-1.673076,-1.685465,-1.672507,-1.708822
201,1,1.771131,0.399404,-0.262140,-0.716557,1.207546,-1.677433,-1.670592,-1.682804,-1.667956,-1.707535
202,1,2.107051,0.474886,-0.223580,-0.754753,1.456540,-1.676217,-1.675720,-1.684459,-1.672709,-1.707885
203,1,1.399442,0.350563,-0.166644,-0.621751,0.932398,-1.668690,-1.660784,-1.673433,-1.657727,-1.699923


In [24]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import itertools
import scipy.stats
from tqdm.notebook import tqdm
import random

In [25]:
model_name = 'power_2011_combat_h{}_l{}_r{}_ep{}'.format(hidden_size, latent_size, int(l2_rate*100), epochs)

model, _ = load_network(model_name)

if torch.cuda.is_available():
    model.cuda()

model.eval()

Successfully loaded model /home/kimyoun/Desktop/VAE_visualization/vae_nets/power_2011_combat_h80_l5_r10_ep1000/power_2011_combat_h80_l5_r10_ep1000_999.nn.


VAE(
  (fc1): Linear(in_features=34980, out_features=80, bias=True)
  (fc21): Linear(in_features=80, out_features=5, bias=True)
  (fc22): Linear(in_features=80, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=80, bias=True)
  (fc4): Linear(in_features=80, out_features=34980, bias=True)
)

In [26]:
with open(os.path.join('latent_visualization','vae_ROIS.txt')) as f:
    roi_list = [s.replace('\n','') for s in f.readlines()]

network_labels = pd.read_csv(os.path.join('latent_visualization', 'power264_networklabels.csv'))
network_labels['NetworkLabel'].unique()

array(['Uncertain', 'Sensory/somatomotor Hand',
       'Sensory/somatomotor Mouth', 'Cingulo-opercular Task Control',
       'Auditory', 'Default mode', 'Memory retrieval?',
       'Ventral attention', 'Visual', 'Fronto-parietal Task Control',
       'Salience', 'Subcortical', 'Cerebellar', 'Dorsal attention'],
      dtype=object)

In [27]:
label_list = network_labels['NetworkLabel'].unique()
label_combinations = list(itertools.combinations_with_replacement(label_list, 2))
label_combinations
label_masks = {}
for label1, label2 in label_combinations:
    ROI_match_filt = (network_labels['NetworkLabel'].str.fullmatch(label1) |
                     network_labels['NetworkLabel'].str.fullmatch(label2))
    ROI_indices = network_labels['ROI'][ROI_match_filt]
    ROI_mat_indices = []
    for x in ROI_indices:
        for y in ROI_indices:
            if (label1 != label2 and x != y) or (label1 == label2):
                ROI_mat_indices.append((x-1,y-1))
    ROI_mat_indices = list(zip(*ROI_mat_indices))
    ROI_mat_indices = tuple(np.array(x) for x in ROI_mat_indices)
    label_mask = np.zeros((264, 264))

    label_mask[ROI_mat_indices] = 1
    label_masks[(label1, label2)] = label_mask

In [28]:
ref = [[(i,j) for i in range(264)] for j in range(264)]

roi_test = np.array(ref)

roi_test_comp = np.array(ref)
roi_test_comp[np.triu_indices(264)] = (-1, -1)

roi_idx = [(int(x.split('-')[0].replace('ROI', ''))-1, int(x.split('-')[1].replace('ROI', ''))-1) for x in roi_list]
roi_idx_zip = tuple(np.array(x) for x in zip(*roi_idx))

roi_test[roi_idx_zip] = (-1, -1)

print(np.all(roi_test == roi_test_comp))
print(len(roi_idx))

True
34980


In [29]:
def latent_diff_mat(model, curr_latent, prev_val, curr_val, dummy_latent_val):
    curr_z = np.ones(model.latent_size)*dummy_latent_val
    prev_z = np.zeros(model.latent_size)*dummy_latent_val

    diff = np.zeros(model.input_size)
    diff_norm = np.zeros(model.input_size)

    curr_z[curr_latent] = curr_val
    prev_z[curr_latent] = prev_val

    curr_z_torch = torch.from_numpy(curr_z).float()
    prev_z_torch = torch.from_numpy(prev_z).float()

    if torch.cuda.is_available():
        curr_z_torch = curr_z_torch.cuda()
        prev_z_torch = prev_z_torch.cuda()

    curr_pred = model.decode(curr_z_torch).data.cpu().numpy()
    prev_pred = model.decode(prev_z_torch).data.cpu().numpy()

    diff = np.abs(curr_pred - prev_pred) * np.abs(curr_val - prev_val)

    diff_mat = np.zeros((264,264))
    upper = np.triu_indices(264)
    diff_mat[upper] = diff
    diff_mat.T[upper] = diff_mat[upper]

    return diff_mat

def latent_mat(model, latent, values, weights=None, latent_orig=None):

    diff_agg = None
    for i in range(len(values)-1):

        if latent_orig is None:
            curr_z = np.zeros(model.latent_size)
            prev_z = np.zeros(model.latent_size)
        else:
            curr_z = np.copy(latent_orig)
            prev_z = np.copy(latent_orig)

        curr_z[latent] = values[i+1]
        prev_z[latent] = values[i]

        curr_z_torch = torch.from_numpy(curr_z).float()
        prev_z_torch = torch.from_numpy(prev_z).float()

        if torch.cuda.is_available():
            curr_z_torch = curr_z_torch.cuda()
            prev_z_torch = prev_z_torch.cuda()

        ##curr_pred = model.decode(curr_z_torch).data.cpu().numpy()
        ##prev_pred = model.decode(prev_z_torch).data.cpu().numpy()
        curr_pred = model.decode(curr_z_torch)
        prev_pred = model.decode(prev_z_torch)

        diff = torch.abs(curr_pred - prev_pred)

        if weights is not None:
            diff_curr = diff * np.abs(values[i+1] - values[i]) * (weights[i] + weights[i+1]) / 2
        else:
            diff_curr = diff * np.abs(values[i+1] - values[i])

        if diff_agg is None:
            diff_agg = diff_curr
        else:
            diff_agg += diff_curr

    return diff_agg.data.cpu().numpy()

def format_latent_mat(latent_agg_mat):
    formatted_mat = np.zeros((264,264))
    upper = np.triu_indices(264)
    formatted_mat[upper] = latent_agg_mat
    formatted_mat.T[upper] = formatted_mat[upper]
    return formatted_mat

# def latent_network_contribution(latent_agg_mat, label_masks):
#     network_contrib_dict = {'Networks':[], 'Contribution':[]}
#     for label_pair, mask in label_masks.items():
#         network_contrib_dict['Networks'].append(label_pair)
#         network_contrib_dict['Contribution'].append(np.sum(np.sum(mask*latent_agg_mat))/np.sum(np.sum(mask)))
#     contrib_df = pd.DataFrame(network_contrib_dict)
#     contrib_df.sort_values(by='Contribution', ascending=False, inplace=True)
#     return contrib_df

def latent_network_contribution(latent_agg_mat, label_masks):
    network_contrib_dict = {'Networks':[], 'Contribution':[]}
    for label_pair, mask in label_masks.items():
        network_contrib_dict['Networks'].append(label_pair)
        network_contrib_dict['Contribution'].append(np.nansum(np.nansum(mask*latent_agg_mat))/np.nansum(np.nansum(mask)))
    contrib_df = pd.DataFrame(network_contrib_dict)
    contrib_df.sort_values(by='Contribution', ascending=False, inplace=True)
    return contrib_df

In [30]:
latent=0
for dummy_latent in np.linspace(-2,2,4):
    for x in np.linspace(-1,1,2):
        diff_mat = latent_diff_mat(model, 0, dummy_latent, x, dummy_latent)

In [31]:
patient_latents = load_latents(model_name, fname_tag='train')

Reading latents from  /home/kimyoun/Desktop/VAE_visualization/vae_nets/power_2011_combat_h80_l5_r10_ep1000/power_2011_combat_h80_l5_r10_ep1000_train_latents.csv


In [32]:
resolution = 20
diag_latent_mats = [None for _ in range(10)]
patient = patient_latents.iloc[0]
means = patient[patient.index.str.contains('mu')].to_numpy().astype(float)
logvars = patient[patient.index.str.contains('logvar')].to_numpy().astype(float)
std_devs = np.sqrt(np.exp(logvars))
for i, (mean,std_dev) in enumerate(zip(means, std_devs)):
    latent_vals = np.linspace(mean - 3 * std_dev, mean + 3 * std_dev, resolution)
    normal_weights = scipy.stats.norm.pdf(latent_vals, loc=mean, scale=std_dev)
    latent_agg_mat = latent_mat(model, i, latent_vals, weights=normal_weights, latent_orig=means)

    if diag_latent_mats[i] is None:
        diag_latent_mats[i] = latent_agg_mat
    else:
        diag_latent_mats[i] += latent_agg_mat

In [33]:
mat_list = diag_latent_mats
for latent in range(5):
    mat = mat_list[latent]
    mat = format_latent_mat(mat)

In [34]:
resolution = 20
diag_latent_mats = [None for _ in range(10)]
patient = patient_latents.iloc[0]
means = patient[patient.index.str.contains('mu')].to_numpy().astype(float)
logvars = patient[patient.index.str.contains('logvar')].to_numpy().astype(float)
std_devs = np.sqrt(np.exp(logvars))
for i, (mean,std_dev) in enumerate(zip(means, std_devs)):
    latent_vals = np.linspace(mean - 3 * std_dev, mean + 3 * std_dev, resolution)
    normal_weights = scipy.stats.norm.pdf(latent_vals, loc=mean, scale=std_dev)
    latent_agg_mat = latent_mat(model, i, latent_vals, weights=normal_weights, latent_orig=means)

    if diag_latent_mats[i] is None:
        diag_latent_mats[i] = latent_agg_mat
    else:
        diag_latent_mats[i] += latent_agg_mat

mat_list = diag_latent_mats
for latent in range(5):
    mat = mat_list[latent]
    mat = format_latent_mat(mat)